In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

# Create a base class for models
Base = declarative_base()

# Define a table as a Python class
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)

# Create the database (SQLite here)
engine = create_engine('sqlite:///example.db')
Base.metadata.create_all(engine)

# Create a session to interact with the DB
Session = sessionmaker(bind=engine)
session = Session()

# Add data
new_user = User(name="Anass", email="anass@example.com")
session.add(new_user)
session.commit()

# Query data
for user in session.query(User).all():
    print(user.name, user.email)

